In [6]:
import os
import sys
from datetime import datetime
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import napari
import numpy as np
import pandas as pd
import tkinter as tk
from IPython.display import display
from PIL import Image
from scipy import ndimage
from scipy.ndimage import median_filter
from skimage import color, filters, io, measure, morphology
from skimage.io import imread, imsave
from skimage.measure import regionprops
from skimage.metrics import mean_squared_error, peak_signal_noise_ratio, structural_similarity
from skimage.morphology import (area_closing, area_opening, ball, black_tophat, closing, dilation, erosion, opening, remove_small_objects, white_tophat)
from skimage.segmentation import mark_boundaries
from sklearn.metrics import mean_squared_error
from tkinter import filedialog, messagebox
from tkinter import ttk as ttk
import cv2
import cc3d
import pandas as pd
import skimage.measure as measure
import matplotlib.pyplot as plt
from skimage import exposure
import trackpy as tp

#########################################
# List of parameters to include in the subfolder name
key_parameters = [
    'time_start_index', 'time_end_index', 'y_start_index', 'y_end_index', 'x_start_index', 'x_end_index',
    'median_filter_frames', 'median_filter_width', 'median_filter_height', 'background_substraction_neighbors',
    'threshold_percentage', 'morph_op_element_size_xy', 'morph_op_element_size_t', 'connected_components_count', 'min_volume_pixel_size'
]

crop_x_start = 0
crop_x_end = None
crop_y_start = 0
crop_y_end = None
crop_t_start = 0
crop_t_end = None

# Initialize global variables for median filtering and background subtraction
apply_median_filter = False
median_filter_frames = 3  # Default value
background_subtraction = False
background_subtraction_neighbors = 3  # Default value

# Create variables for entry fields
x_start_entry = None
x_end_entry = None
y_start_entry = None
y_end_entry = None
t_start_entry = None
t_end_entry = None
median_filter_checkbox = None
background_subtraction_checkbox = None


default_parameters = {
    'time_start': 0,
    'time_end': -1, 
    'y_start': 0,
    'y_end': -1,
    'x_start': 0,
    'x_end': -1,
    'threshold': 7000,
    'median_filter_size': (5, 3, 3),
    'BKG_Neighbors' : 660,
    'Connectivity' : 26,
    'Structure Element' : (5, 2, 2),
    'Area Size' : 100,
    'trackpy_memory': 2,
    'trackpy_search_range': 3,
    'trackpy_threshold': 3,
}

  
astrocyte3d = None
automatic_parameters = {
    'time_start': 0,
    'time_end': -1,
    'y_start': 0,
    'y_end': -1,
    'x_start': 0,
    'x_end': -1,
    'threshold': 7000,
    'median_filter_size': (5, 3, 3),
    'BKG_Neighbors': 660,
    'Connectivity': 26,
    'Structure_Element': (5, 2, 2),
    'Area_Size': 100,
    'trackpy_memory': 2,
    'trackpy_search_range': 3,
    'trackpy_threshold': 3,
}

# Centroids
def find_centroids(label_stack):
    centroids = []
    for frame in range(label_stack.shape[0]):
        # Measure region properties in the current frame
        regions = measure.regionprops(label_stack[frame])
        centroids_frame = np.array([prop.centroid for prop in regions])
        centroids.append(centroids_frame)
    return centroids

def cumulative_color_objects(label_sequence, history_depth=5):
    num_frames = label_sequence.shape[0]
    object_max_areas = {}
    object_colors = {}

    # Find the frame with max area for each object
    for frame_idx in range(num_frames):
        frame = label_sequence[frame_idx, :, :]
        objects_in_frame = np.unique(frame)
        for obj in objects_in_frame:
            if obj == 0:  # Skip background
                continue
            obj_area = np.sum(frame == obj)
            if obj not in object_max_areas or obj_area > object_max_areas[obj][1]:
                object_max_areas[obj] = (frame_idx, obj_area)
                
    # Create a colormap for each object using the 'plasma' colormap
    for obj, (max_frame, max_area) in object_max_areas.items():
        object_colors[obj] = plt.get_cmap('plasma')
        
    # Prepare a list to hold the colored overlays
    colored_frames = [np.zeros((*label_sequence.shape[1:], 3), dtype=float) for _ in range(num_frames)]

    # Now apply the colors cumulatively
    # Now apply the colors cumulatively
    for frame_idx in range(num_frames):
        current_frame = label_sequence[frame_idx, :, :]
        for obj in np.unique(current_frame):
            if obj == 0:
                continue
            for h in range(history_depth):
                past_frame_idx = max(frame_idx - h, 0)
                past_frame = label_sequence[past_frame_idx, :, :]
                obj_mask = past_frame == obj
                relative_area = np.sum(obj_mask) / object_max_areas[obj][1]
                color_idx = int(relative_area * 256)  # Adjusted to match colormap index range
                color = np.array(object_colors[obj](color_idx)[:3])
                alpha = 1 - (h / history_depth)  # Decrease opacity for past stages
                # Blend the current object's color with its past color
                colored_frames[frame_idx][obj_mask] = (1 - alpha) * colored_frames[frame_idx][obj_mask] + alpha * color

    return colored_frames

#hsv to rgb
def hsv_to_rgb(h, s, v):  # TODO: remove colorsys dependency
    """
    Converts a color from HSV to RGB

    Parameters
    ----------
    h : float
    s : float
    v : float

    Returns
    -------
    numpy.ndarray
        The converted color in RGB space.
    """
    i = np.floor(h*6.0)
    f = h * 6 - i
    p = v * (1 - s)
    q = v * (1 - s * f)
    t = v * (1 - s * (1 - f))
    i = i % 6

    if i == 0:
        rgb = (v, t, p)
    elif i == 1:
        rgb = (q, v, p)
    elif i == 2:
        rgb = (p, v, t)
    elif i == 3:
        rgb = (p, q, v)
    elif i == 4:
        rgb = (t, p, v)
    else:
        rgb = (v, p, q)

    return np.array(rgb, dtype=np.float32)


#Get distinct colors
def get_distinct_colors(n, min_sat=.5, min_val=.5):
    """
    Generates a list of distinct colors, evenly separated in HSV space.

    Parameters
    ----------
    n : int
        Number of colors to generate.
    min_sat : float
        Minimum saturation.
    min_val : float
        Minimum brightness.

    Returns
    -------
    numpy.ndarray
        Array of shape (n, 3) containing the generated colors.

    """
    huePartition = 1.0 / (n + 1)
    hues = np.arange(0, n) * huePartition
    saturations = np.random.rand(n) * (1-min_sat) + min_sat
    values = np.random.rand(n) * (1-min_val) + min_val
    return np.stack([hsv_to_rgb(h, s, v) for h, s, v in zip(hues, saturations, values)], axis=0)

#Colorize segmentation
def colorize_segmentation(seg, ignore_label=None, ignore_color=(0, 0, 0)):
    """
    Randomly colorize a segmentation with a set of distinct colors.

    Parameters
    ----------
    seg : numpy.ndarray
        Segmentation to be colorized. Can have any shape, but data type must be discrete.
    ignore_label : int
        Label of segment to be colored with ignore_color.
    ignore_color : tuple
        RGB color of segment labeled with ignore_label.

    Returns
    -------
    numpy.ndarray
        The randompy colored segmentation. The RGB channels are in the last axis.
    """
    assert isinstance(seg, np.ndarray)
    assert seg.dtype.kind in ('u', 'i')
    if ignore_label is not None:
        ignore_ind = seg == ignore_label
    seg = seg - np.min(seg)
    colors = get_distinct_colors(np.max(seg) + 1)
    np.random.shuffle(colors)
    result = colors[seg]
    if ignore_label is not None:
        result[ignore_ind] = ignore_color
    return result



#Crop Dataset
def crop_dataset(data, crop_params):
    time_start, time_end = crop_params['time_start'], crop_params['time_end']
    y_start, y_end = crop_params['y_start'], crop_params['y_end']
    x_start, x_end = crop_params['x_start'], crop_params['x_end']
    
    if time_end == -1: time_end = data.shape[0]
    if y_end == -1: y_end = data.shape[1]
    if x_end == -1: x_end = data.shape[2]
    
    return data[time_start:time_end, y_start:y_end, x_start:x_end]

#Load Dataset
def load_dataset():
    global astrocyte3d, automatic_parameters, input_file_path
    file_path = filedialog.askopenfilename(title="Select Dataset", filetypes=[("TIFF Files", "*.tif"), ("All Files", "*.*")])
    if file_path:
        input_file_path = file_path  # Store the input file path
        astrocyte3d = imread(file_path)
        messagebox.showinfo("Info", "Dataset loaded successfully.")

        # Update cropping parameters to the full dimensions of the loaded dataset
        t_dim, y_dim, x_dim = astrocyte3d.shape
        automatic_parameters.update({
            'time_end': t_dim,
            'y_end': y_dim,
            'x_end': x_dim
        })

#Open Parameter window
def open_auto_params_dialog():
    global astrocyte3d, automatic_parameters

    dialog = tk.Toplevel(root)
    dialog.title("Automatic Processing Parameters")

    # Use a dictionary to hold the entry fields corresponding to each parameter
    param_entries = {}

    # Function to populate the dialog with entry fields
    def create_param_entry(param_name, default_value):
        frame = tk.Frame(dialog)
        label = tk.Label(frame, text=f"{param_name}:")
        entry = tk.Entry(frame)
        entry.insert(0, str(default_value))
        label.pack(side=tk.LEFT)
        entry.pack(side=tk.LEFT)
        frame.pack()
        param_entries[param_name] = entry

    # Create entry fields for all parameters including cropping dimensions
    for param, value in automatic_parameters.items():
        create_param_entry(param, value)

    # Function to reset parameters to default values
    def use_default_parameters():
        for param, value in default_parameters.items():
            param_entries[param].delete(0, tk.END)
            param_entries[param].insert(0, str(value))

    # Function to apply parameters
    def apply_parameters():
        global automatic_parameters
        automatic_parameters['trackpy_memory'] = int(param_entries['trackpy_memory'].get())
        automatic_parameters['trackpy_search_range'] = int(param_entries['trackpy_search_range'].get())
        automatic_parameters['trackpy_threshold'] = int(param_entries['trackpy_threshold'].get())
        for param in automatic_parameters:
            entry_value = param_entries[param].get()
            if param in ['Structure_Element', 'median_filter_size']:
                values = entry_value.strip('()').split(',')
                automatic_parameters[param] = tuple(map(int, values))
            elif param in ['time_start', 'time_end', 'y_start', 'y_end', 'x_start', 'x_end']:
                automatic_parameters[param] = int(entry_value) if entry_value != '-1' else -1
            else:
                automatic_parameters[param] = type(automatic_parameters[param])(entry_value)
        dialog.destroy()
        automatic_image_processing_steps()

    apply_button = tk.Button(dialog, text="Apply", command=apply_parameters)
    apply_button.pack()

    default_button = tk.Button(dialog, text="Use Default", command=use_default_parameters)
    default_button.pack()

    dialog.transient(root)  
    dialog.grab_set()     
    root.wait_window(dialog)  
    
#Pipeline Steps  
def automatic_image_processing_steps():
    global astrocyte3d, automatic_parameters, progress, input_file_path
    if astrocyte3d is None:
        messagebox.showwarning("Warning", "No dataset loaded.")
        return

    # Create output directory in the same location as input file
    output_dir = os.path.join(os.path.dirname(input_file_path), 'output')
    png_dir = os.path.join(output_dir, 'PNGs')  
    os.makedirs(png_dir, exist_ok=True)

    def save_image(image, name, format="tif"):
        if image.dtype == np.float32:  # Check if the data type is float32
            image = (image * 255).astype(np.uint8)  # Scale and convert to uint8
        if format == "tif":
            filename = os.path.join(output_dir, f"{name}.{format}")
            imsave(filename, image)
        elif format == "png":
            for i, frame in enumerate(image):
                filename = os.path.join(png_dir, f"{name}_frame_{i:03d}.png")
                skimage.io.imsave(filename, frame)

    # Crop dataset
    astrocyte3d = crop_dataset(astrocyte3d, automatic_parameters)
    
    progress['value'] = 20  # Update progress after a step
    root.update_idletasks()  # Refresh GUI to reflect the change

    # Apply Median Filtering
    footprint = np.ones(automatic_parameters['median_filter_size'])
    astromedian = median_filter(astrocyte3d, footprint=footprint)
    #save_image(astromedian, "Median_Filtered", format="tif")
    
    progress['value'] = 30  # Update progress after a step
    root.update_idletasks()  # Refresh GUI to reflect the change

    # Calculate the Background
    Nframes = automatic_parameters['BKG_Neighbors']
    frames = astromedian.shape[0]
    bg_image_median = np.zeros(astromedian.shape, 'uint16')
    for i in range(frames):
        if i - Nframes < 0:
            bg_image_median[i,:,:] = np.min(astromedian[0:(i+Nframes+1),:,:], axis=0)
        elif i + Nframes + 1 > frames:
            bg_image_median[i,:,:] = np.min(astromedian[(i-Nframes):,:,:], axis=0)
        else:
            bg_image_median[i,:,:] = np.min(astromedian[(i-Nframes):(i+Nframes+1),:,:], axis=0)

    # Background Subtraction
    diffI_Md_MdBKG = astromedian - bg_image_median
    #save_image(diffI_Md_MdBKG, "Background Substracted", format="tif")
    #save_image(bg_image_median, "Background", format="tif")
    
    progress['value'] = 40  # Update progress after a step
    root.update_idletasks()  # Refresh  GUI to reflect the change

    # Apply Threshold
    t = automatic_parameters['threshold']
    Threshold = (diffI_Md_MdBKG > t)
    #save_image(Threshold, "Thresholded", format="tif")
    
    progress['value'] = 50  # Update progress after a step
    root.update_idletasks()  # Refresh GUI to reflect the change

    # Apply Connected Components Labeling
    Con = automatic_parameters['Connectivity']
    Labelled = cc3d.connected_components(Threshold, connectivity=Con)
    col = colorize_segmentation(Labelled, ignore_label=0)
    
    progress['value'] = 60  # Update progress after a step
    root.update_idletasks()  # Refresh GUI to reflect the change

    # Fill Holes
    se = np.ones(automatic_parameters['Structure_Element'])
    Closing = ndimage.binary_closing(Threshold, se).astype(Threshold.dtype)
    #save_image(Closing, "Holes Filled", format="png")
    
    progress['value'] = 70  # Update progress after a step
    root.update_idletasks()  # Refresh GUI to reflect the change

    # Restrict Area Size
    Area = automatic_parameters['Area_Size']
    Area_Restrict = remove_small_objects(Labelled, Area)
    
    progress['value'] = 80  # Update progress after a step
    root.update_idletasks()  # Refresh GUI to reflect the change

    # Labeling
    Labelled = cc3d.connected_components(Area_Restrict, connectivity=Con)
    col = colorize_segmentation(Labelled, ignore_label=0)
    #save_image(col, "Area Restricted", format="png")
    
    regions = measure.regionprops(Labelled)
    properties_extracted = True  # Flag 

    if not regions:
        messagebox.showwarning("Warning", "No regions found in the labeled image.")
        properties_extracted = False

    region_properties_list = []

    if properties_extracted:
        # Loop through each region 
        for region in regions:
            properties = {
                'Label': region.label,
                'Area': region.area,
                'Centroid': region.centroid,
                'BoundingBox': region.bbox,
            }
            region_properties_list.append(properties)

        if not region_properties_list:
            messagebox.showwarning("Warning", "No region properties were extracted.")
            properties_extracted = False

    # Proceed 
    if properties_extracted:
        region_properties_df = pd.DataFrame(region_properties_list)
        region_properties_df.set_index('Label', inplace=True)
        # Save to an Excel file
        output_excel_file = 'region_properties.xlsx'
        region_properties_df.to_excel(output_excel_file)
        messagebox.showwarning("Warning", f"Region properties saved to {output_excel_file}")

    # Calculate Boundary Masks
    num_slices = Labelled.shape[0]
    boundary_masks = np.zeros_like(Labelled, dtype=bool)

    for slice_idx in range(num_slices):
        labeled_slice = Labelled[slice_idx]
        boundaries = mark_boundaries(np.zeros_like(labeled_slice), labeled_slice)
        boundary_mask = (boundaries[:, :, 0] > 0)
        boundary_masks[slice_idx] = boundary_mask
        
    # Generate colored sequence
    colored_sequence = cumulative_color_objects(Labelled, history_depth=5)

    # Stack the frames to create a 4D array (time, x, y, color channels)
    colored_stack = np.stack(colored_sequence, axis=0)
    
    progress['value'] = 90  # Update progress after a step
    root.update_idletasks()  # Refresh GUI to reflect the change
    
    # Find centroids in each frame of the labeled stack
    centroids = find_centroids(Labelled)

    # Prepare DataFrame for trackpy
    frames = []
    for frame_number, centroid_frame in enumerate(centroids):
        for yx in centroid_frame:
            frames.append({'y': yx[0], 'x': yx[1], 'frame': frame_number})

    df = pd.DataFrame(frames)

    # Check if DataFrame is suitable for trackpy
    if not df.empty and 'x' in df.columns and 'y' in df.columns and 'frame' in df.columns:
        # Use trackpy to link features into particle trajectories
        t = tp.link(df, search_range=automatic_parameters['trackpy_search_range'], memory=automatic_parameters['trackpy_memory'])

        # Filter out short-lived features
        t1 = tp.filter_stubs(t, threshold=automatic_parameters['trackpy_threshold'])

        unique_particles = t1['particle'].unique()

        # Check if there are any unique particles
        if unique_particles.size > 0:
            # Create a maximum projection of the labeled dataset
            max_projection = np.max(Labelled, axis=0)

            # Normalize for better visualization
            normalized_max_projection = exposure.rescale_intensity(max_projection, in_range='image', out_range=(0, 255))

            # Plot trajectories on the maximum projection
            fig, ax = plt.subplots(figsize=(10, 10))
            ax.imshow(normalized_max_projection, cmap='gray')

            # Plot each trajectory
            for particle in unique_particles:
                trajectory = t1[t1['particle'] == particle]
                ax.plot(trajectory['x'], trajectory['y'], marker='o', markersize=3, linestyle='-', linewidth=1)

            # Remove axes and save the plot
            ax.axis('off')
            plt.tight_layout()
            plt.savefig('trajectory_plot.png', dpi=300)
            plt.close(fig)
            messagebox.showwarning("Warning", f"Trajectory plot saved as trajectory_plot.png")

    else:
        messagebox.showwarning("Warning", "DataFrame is empty or not properly formatted for trackpy.")
        unique_particles = []

    progress['value'] = 100  # Set to 100% when processing is complete
    root.update_idletasks()

    # Open results in Napari
    with napari.gui_qt():
        viewer = napari.Viewer()
        viewer.add_image(astrocyte3d, name='Original Data')
        viewer.add_image(astromedian, name='Median Filtered')
        viewer.add_image(bg_image_median, name='Background')
        viewer.add_image(diffI_Md_MdBKG, name='Background Subtracted')
        viewer.add_image(Threshold, name='Thresholded')
        viewer.add_image(Closing, name='Holes Filled')
        viewer.add_image(col, name='Final Labelled')
        viewer.add_labels(boundary_masks, name='Boundary Masks')
        viewer.add_image(colored_stack, name='Colored Sequence')
        #viewer.add_image(trajectory_image, name='Trajectories on Max Projection')

    progress['value'] = 0  # Reset the progress bar after processing


def create_main_window():
    global root, progress  
    root = tk.Tk()
    root.title("Automatic Image Processing")

    load_button = tk.Button(root, text="Load Dataset", command=load_dataset)
    load_button.pack()

    params_button = tk.Button(root, text="Set Parameters & Process", command=open_auto_params_dialog)
    params_button.pack()

    #process_button = tk.Button(root, text="Start Processing", command=automatic_image_processing_steps)
    #process_button.pack()
    
    # Create progress bar
    progress = ttk.Progressbar(root, orient=tk.HORIZONTAL, length=200, mode='determinate')
    progress.pack()

    return root

if __name__ == '__main__':
    root = create_main_window()
    root.mainloop()


Frame 347: 1 trajectories present.


C:\Users\User\anaconda3\envs\napari-env\lib\site-packages\napari\_qt\qt_event_loop.py:265: FutureWarning: 
The 'gui_qt()' context manager is deprecated.
If you are running napari from a script, please use 'napari.run()' as follows:

    import napari

    viewer = napari.Viewer()  # no prior setup needed
    # other code using the viewer...
    napari.run()

In IPython or Jupyter, 'napari.run()' is not necessary. napari will automatically
start an interactive event loop for you: 

    import napari
    viewer = napari.Viewer()  # that's it!

  warn(
